Author: Alkistis Pourtsidou, ICG Portsmouth 

Using part of http://camb.readthedocs.io/en/latest/CAMBdemo.html

To run this notebook you need to have CAMB and the CAMB python package installed. 
In order to install the CAMB python package on your computer follow the instructions in http://camb.readthedocs.io/en/latest/

In [1]:
%matplotlib inline
import sys, platform, os

from matplotlib import pyplot as plt
import numpy as np
#print('Using CAMB installed at '+ os.path.realpath(os.path.join(os.getcwd(),'..')))
#sys.path.insert(0,os.path.realpath(os.path.join(os.getcwd(),'..')))
import camb
from camb import model, initialpower

In [2]:
import scipy
from scipy.interpolate import interp1d
from __future__ import division

In [3]:
from scipy import integrate
from scipy import linalg

pi=np.pi

In [4]:
#Fiducial cosmological parameters
c=3e5
hubble=0.678
omegab=0.022*pow(hubble,-2)
omegac=0.119*pow(hubble,-2)
om0=omegac+omegab
H00=100*hubble
Ass=2.14e-9
nss = 0.968

gamma=0.545

print om0

0.306732450988


In [5]:
#Set up the fiducial cosmology
pars = camb.CAMBparams()
#Set cosmology
pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=nss, r=0, As=Ass)
pars.set_for_lmax(2500, lens_potential_accuracy=0);

In [6]:
#calculate results for these parameters
results = camb.get_results(pars)

In [7]:
#Get matter power spectrum at z=0: P(k,z=0)

#Not non-linear corrections couples to smaller scales than you want
pars.set_matter_power(redshifts=[0.], kmax=5.0)

#Non-Linear spectra (Halofit)
pars.NonLinear = model.NonLinear_both
results.calc_power_spectra(pars)
kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=5.0, npoints = 200)

In [8]:
#Construct P(k,z=0) interpolating function, in units of Mpc (no h)
Pkz0 = interp1d(kh*hubble, pk[0]/pow(hubble,3))

In [9]:
#Redshift bins
zlist = np.arange(0.7,1.45,0.1)
ztest = zlist[0]
Nzbins = len(zlist)

#mean number density of galaxies
factor = pow(hubble,3)*1e-4
nbarlist = [17.5, 19, 18, 17, 15, 13, 12, 10]

print zlist
print "ztest =", ztest
print "Number of redshift bins =", Nzbins

[ 0.7  0.8  0.9  1.   1.1  1.2  1.3  1.4]
ztest = 0.7
Number of redshift bins = 8


In [10]:
#Define E(z) = H(z)/H0
def Ez(zc):
    return np.sqrt(1-om0+om0*pow(1+zc,3))

#Define the comoving distance
def drdz(zp):
    return (c/H00)/Ez(zp)
def rcom(zc):
    return scipy.integrate.romberg(drdz,0,zc)

print rcom(ztest)

2580.87137613


In [11]:
#Define the growth function in LCDM
def get_growth(zz):
    omz=om0*pow(1+zz,3)/(om0*pow(1+zz,3)+1-om0)
    return pow(omz,gamma)

print get_growth(ztest)

0.813619628081


In [12]:
#Get the growth factor 
def Dg_dz(zz):
    return get_growth(zz)/(1+zz)
def Dgz(zc):
    ans = scipy.integrate.romberg(Dg_dz, 0.0, zc)
    return np.exp(-ans)

print Dgz(ztest)

0.69801369479


In [26]:
#fiducial bHI from Bull et al 2015
def bHI(zc):
    return 0.67+0.18*zc+0.05*pow(zc,2)

#fiducial bgal
def bg(zc):
    return np.sqrt(1+zc)

#fiducial OmHI Mario's fit
def OmHI(zc):
    return 0.00048+0.00039*zc-0.000065*pow(zc,2)

#mean brightness temperature [mK] Mario's fit
def Tb(zc):
    return 0.0559+0.2324*zc-0.024*pow(zc,2)

print Tb(ztest)

0.20682


In [14]:
#Construct PHIg(k,μ,z) 
def PHIg(kk,mu,zc):    
    return Tb(zc)*(bHI(zc)+get_growth(zc)*mu**2)*(bg(zc)+get_growth(zc)*mu**2)*pow(Dgz(zc),2)*Pkz0(kk)

In [15]:
#Construct P_HI(k,μ,z) [mK^2]
def PHI(kk,mu,zc):
    return pow(Tb(zc),2)*pow(bHI(zc),2)*pow(1+(get_growth(zc)/bHI(zc))*mu**2,2)*pow(Dgz(zc),2)*Pkz0(kk)

#Construct Pgg(k,μ,z) 
def Pgg(kk,mu,zc):    
    return pow(bg(zc),2)*pow(1+(get_growth(zc)/bg(zc))*mu**2,2)*pow(Dgz(zc),2)*Pkz0(kk)

In [16]:
#SKA1 noise specs
Ndishes=130
Ddish=15*100 #cm
Nbeams=1

def thetab(zc):
    return 21*(1+zc)/Ddish

def omegapix(zc):
    return 1.13*pow(thetab(zc),2)

Area=7000.0 #deg^2
omegatot = Area*pow(pi/180,2)
ttotal = 4000*60*60 #4000 hours

Tsys = 25.0*1e3 #mK

def tobs(zc):
    return ttotal*(omegapix(zc)/omegatot)*Ndishes*Nbeams

Dzbin = 0.1
dfpix = 50*1e3 #Hz
midfreq = 1420.4e6 #Hz

def dzpix(zc):
    return pow(1+zc,2)*dfpix/midfreq
def sigpix(zc,Tsys):
    return Tsys/np.sqrt(dfpix*tobs(zc)) 
def dVpixdz(zz):    
    return c*pow(rcom(zz),2)/(H00*Ez(zz))
def Vpix(zc):
    return omegapix(zc)*scipy.integrate.romberg(dVpixdz,zc-dzpix(zc)/2,zc+dzpix(zc)/2)

def Wsq(kk,zc):
    return np.exp(-pow(kk,2)*pow(rcom(zc),2)*pow(thetab(zc),2)/(8*np.log(2)))

def Pnoise(kk,zc,Tsys):
    return pow(sigpix(zc,Tsys),2)*Vpix(zc)*pow(Wsq(kk,zc),-1.)

In [17]:
def kmin(zc):
    return 2*pi/np.sqrt(pow(rcom(zc),2)*omegatot)

def kmax(zc):
    return 0.14*pow(1+zc,2/(2+nss)) #non-linear cutoff (Smith et al 2003)

In [18]:
#survey (bin) volume [Mpc^3]
def dVsurdz(zz):    
    return omegatot*c*pow(rcom(zz),2)/(H00*Ez(zz))
    
def Vsur(zc):
    return scipy.integrate.romberg(dVsurdz,zc-Dzbin/2,zc+Dzbin/2)

def Pshot(zc):
    return 1/nbar

def Veff(kk,mu,zc):
    return Vsur(zc)*(pow(PHIg(kk,mu,zc),2)/(pow(PHIg(kk,mu,zc),2)+(PHI(kk,mu,zc)+Pnoise(kk,zc,Tsys))
                                         *(Pgg(kk,mu,zc)+Pshot(zc))))

In [19]:
Npar = 4
params = ["0:fsig8","1:bsig8","2:DA","3:H"]

In [21]:
#Fisher matrix derivatives

def dlnP_dlnfsig8(kk,mu,zc):
    return ((mu**2*get_growth(zc)/(bg(zc)+mu**2*get_growth(zc)))+
           (mu**2*get_growth(zc)/(bHI(zc)+mu**2*get_growth(zc))))

def dlnP_dlnbHIsig8(kk,mu,zc):
    return bHI(zc)/(bHI(zc)+mu**2*get_growth(zc))

def dlnP_dlnDA(kk,mu,zc):
    dk = (kmax(zc)-kmin(zc))/400
    return (-2.0+2*mu**2*(1-mu**2)*get_growth(zc)/(bHI(zc)+mu**2*get_growth(zc))
            +2*mu**2*(1-mu**2)*get_growth(zc)/(bg(zc)+mu**2*get_growth(zc))
            -kk*(1-mu**2)*(1/Pkz0(kk))*(Pkz0(kk+dk)-Pkz0(kk-dk))/(2*dk))

def dlnP_dlnH(kk,mu,zc):
    dk = (kmax(zc)-kmin(zc))/400
    return (1.0+2*mu**2*(1-mu**2)*get_growth(zc)/(bHI(zc)+mu**2*get_growth(zc))
            +2*mu**2*(1-mu**2)*get_growth(zc)/(bg(zc)+mu**2*get_growth(zc))
            +kk*mu**2*(1/Pkz0(kk))*(Pkz0(kk+dk)-Pkz0(kk-dk))/(2*dk))

In [22]:
def dF(kk,mu):
    return (1./(8*pi*pi))*pow(kk,2)*deriv_i(kk,mu,zc)*deriv_j(kk,mu,zc)*Veff(kk,mu,zc)    

In [23]:
#2D integration function
def integrate2D(dfun, kgrid, mugrid):
    
    muint = [scipy.integrate.simps(dfun.T[i], mugrid) for i in range(kgrid.size)]
    return scipy.integrate.simps(muint, kgrid)

In [24]:
mugrid = np.linspace(-1., 1., 200) 

In [27]:
#%%time
#   Fisher matrix   # 

Npar = 4
#create array of zeros
s = (Npar,Npar)

for zi in range(0,Nzbins):
    zc = zlist[zi]
    nbar = factor*nbarlist[zi]
    #print Pshot(zc)
    kgrid = np.linspace(kmin(zc), kmax(zc), 400)
    K, MU = np.meshgrid(kgrid, mugrid)
    Fishermat = np.zeros(s)
    for i in range(0,Npar):  
        def deriv_i(kk,mu,zc):
            if i==0:  return dlnP_dlnfsig8(kk,mu,zc)
            elif i==1:  return dlnP_dlnbHIsig8(kk,mu,zc)
            elif i==2:  return dlnP_dlnDA(kk,mu,zc)
            elif i==3:  return dlnP_dlnH(kk,mu,zc)
            else: print "index out of range"
        for  j in range(0,Npar):
            if j>=i:
                def deriv_j(kk,mu,zc):
                    if j==0:  return dlnP_dlnfsig8(kk,mu,zc)
                    elif j==1:  return dlnP_dlnbHIsig8(kk,mu,zc)
                    elif j==2:  return dlnP_dlnDA(kk,mu,zc)
                    elif j==3:  return dlnP_dlnH(kk,mu,zc)
                    else: print "index out of range" 
                Fishermat[i][j] = integrate2D(dF(K,MU),kgrid,mugrid)                
            else: Fishermat[i,j] = Fishermat[j,i]
    print zc, np.sqrt(linalg.inv(Fishermat)[0,0]),\
              np.sqrt(linalg.inv(Fishermat)[1,1]),\
              np.sqrt(linalg.inv(Fishermat)[2,2]),\
              np.sqrt(linalg.inv(Fishermat)[3,3])

0.7 0.0542082939488 0.0650623823189 0.0345929021997 0.0409447509186
0.8 0.0609596509171 0.075267267719 0.0393352562848 0.0464380812683
0.9 0.0704094314554 0.0921550746335 0.0476750270158 0.0541378558098
1.0 0.0876375805843 0.124624944445 0.0630075926336 0.0687185315028
1.1 0.114924747727 0.174558555984 0.0847202247872 0.0915641979376
1.2 0.141130693789 0.21953131177 0.103046928108 0.113998572186
1.3 0.155088519033 0.240232879772 0.111182233848 0.127069619406
1.4 0.163150814617 0.249326603185 0.114879666628 0.13503094401
